In [9]:
import numpy as np
from numpy.random import choice
from numpy.random import multinomial
from numpy.random import dirichlet
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 


In [ ]:
# utils

class DocTermMatrix(object):
	def __init__(self, corpus, **kwargs):
		'''
		corpus: a collection of raw documents
		with **kwargs you can pass arguments of CountVectorizer
		creates a doc-term matrix instance (as scipy sparse matrix)
		'''
		
		vectorizer = CountVectorizer(**kwargs)
		self.num_docs = len(corpus)
		self.vocab_size = len(vectorizer.vocabulary_)
		self.shape = (self.num_docs, self.vocab_size)
    	self.matrix = vectorizer.fit_transform(docs)
	
	def dataframe(self):
		'''
		returns the doc-term matrix as pandas dataframe
		'''
		return pd.DataFrame(x1.toarray(), columns = vectorizer.get_feature_names())

	def length(self):
		"""
		Calculates the number of words in each document, i.e. its length.
		Returns an array of the same size as the corpus.
		"""
		# lengths_list = np.zeros(self.row_dim)
		lengths_list = self.df().apply(lambda x: len(x.nonzero()[0]), axis=1)
		return lengths_list

	@classmethod
	def word_index(cls, doc_row):
		"""
		doc_row: document vector (of size V, vocab-size)

		Returns an array of repeated indices/terms for the words in the document;
		the indices/terms are repeated the same number of times as the number of occurrences for the corresponding word;
		the length of the array is thus equal to the document length.
		"""
		# doc_row = np.array(doc_row)
		for idx in doc_row.nonzero()[0]:
        	for i in range(int(doc_row[idx])):
            	yield idx

# My journey through hLDA:

Below, firstly, is my implementation of the hierarchical prior for the collapsed Gibbs sampler, largely following the notation of the notes.

I have my questions in the code, but here is an overarching one:
    - I can understand how, by implementing the Polya urn scheme with probabilities that reward previously drawn topics ("rich get richer"?), we would get a few topics being general and others more specific. It is not obvious to me where we made sure that there is a single, unique topic at the root of every path through the hierarchy. [which is how it is plotted in Blei et al. 2004 - 'Hierarchical Topic Models and the Nested Chinese Restaurant Process']

In [ ]:
class hLDA(object):
	def __init__(self, tree_depth, alpha, gamma, eta):
		'''
		gamma: the parameter for the (base) Dirichlet from which the prior vector ('m') is drawn.
				in the notes' notation, this is alpha'.
		tree_depth: a parameter for the number of levels in a hierarchy.
		'''
		self.gamma = gamma
		self.alpha = alpha
		self.k = tree_depth
		self.eta = eta


	def priors(self, dtm, seed=None):
		self.num_docs, self.vocab_size = dtm.shape

		# number of times topic K allocaition variables generate term/word V
		self.nkv = np.zeros((self.num_topics, self.vocab_size))

		#### dtm = DocTermMatrix(corpus).dataframe()
		# Document-specific vectors that record the multiplicity of each label
		self.local_count = np.zeros((self.num_docs, self.k))
		# Global vector which keeps tracks of all draws from the base measure
		self.global_count = np.zeros(self.k)

		if seed is not None:
			np.random.seed(seed)

		# Simulating from the hierarchical prior for the first document
		sample = {}
		sample[0] = multinomial(1,[1/self.k]*self.k).nonzero()[0][0] # sample s1
		R = set(sample.values())
		M = 1	
		z = {}
		# set the topic of the first word in the first doc to s1
		z[(0,0)] = sample[0]
		self.global_count[0, sample[0]] += 1
		self.local_сount[0, sample[0]] += 1
		for n, w in enumerate(DocTermMatrix.word_index(dtm[0,:])):
			draw = hLDA.roll_die_first_doc()
			if draw is not None and draw <= self.k:
				z[(0,n+1)] = draw
				self.local_count[0, draw] += 1
			elif draw > self.k and draw <= 2*self.k:
				sample[M+1] = draw - self.k # assign the s_{Mn+1} = s

				# update local and global count?
				self.local_count[0, sample[M+1]] += 1
				self.global_count[0, sample[M+1]] += 1

				M += 1
				# set z_{n+1} equal to?
				z[(0, n+1)] = sample[M+1]
			elif draw > 2*self.k:
				new_s = multinomial(1,[1/self.k]*self.k).nonzero()[0][0]
				sample[M+1] = new_s
				M += 1
				self.global_count[0, new_s] += 1
				self.local_count[0, new_s] += 1
				# set z_{n+1} equal to?
				z[(0, n+1)] = new_s
			R = set(sample)

		# hierarchical prior across documnents
		for idx, bow in dtm[1:].iterrows(): # bow = bag-of-words
			for n, w in enumerate(DocTermMatrix.word_index(bow)):
				if n == 0:
					draw = hLDA.roll_die_first_word()
					if draw <= self.k:
						z[(idx,n)] = draw # double-check
						self.local_count[idx, draw] = 1
		# do we increment counts for M_{k,v} here too? how does this affect the prior of beta vs. usual case?
						#self.nkv[z[(idx,n)],w] += 1
					else:
						sample[M+1] = multinomial(1,[1/self.k]*self.k).nonzero()[0][0]
						z[(idx,n)] = sample[M+1] # is this the s in the notes?
						M += 1
						#self.nkv[z[(d,i)],w] += 1
				else:
					draw = hLDA.roll_die_nth_word()
					if draw <= self.k:
						# need to align n's
						z[(idx, n+1)] = draw
						self.global_count[idx, draw] += 1
						self.local_count[idx, draw] += 1
						#self.nkv[z[(idx,n+1)],w] += 1

					else:
						sample[M+1] = multinomial(1,[1/self.k]*self.k).nonzero()[0][0]
						z[(idx,n+1)] = sample[M+1]
						self.global_count[idx, sample[M+1]] += 1
						self.local_count[idx, sample[M+1]] += 1
						M += 1
						#self.nkv[z[(idx,n+1)],w] += 1
		return M, z

	def collapsed_gibbs(self, dtm, maxiter=100):
		'''
		dtm: document-term matrix of shape [n_docs, n_vocab]
		'''
		M, z = self.priors(dtm)
		for iterr in range(maxiter):
			for d in range(self.num_docs):
				for i, w in enumerate(DocTermMatrix.word_index(dtm[d,:])):
					self.update_counts(True, d, i, w)
					z[(d,i)] = self.sample_topic_assignment(d, w, M)
					self.update_counts(False, d, i, w)
		# can pick a sample that maximises joint probability
		# and recover estimates of theta and beta from there
		
		return z


	def sample_topic_assignment(self, d, w, M):
		# THETA and BETA have been integrated out and so do not enter as parameters.
		
		zdn_probs = np.zeros(self.k)
		for k in range(self.k):
			# self.nkv.shape[1] = column dimension of self.nkv = vocab_size
			prior = (self.local_count[d, k] + (self.alpha/(self.gamma+M))*(self.global_count[k] + self.gamma/K))
			prior /= (self.alpha + sum(self.local_count[d,:]))
			likelihood = (self.nkv[k,w] + self.eta/self.vocab_size) / (self.nkv[k, :].sum() + self.eta)

			zdi_probs[k] = prior * likelihood
		# normalize probabilities
		zdn_probs /= sum(zdi_probs)
		zdn = multinomial(1, zdn_probs).nonzero()[0][0]
		return zdn

	def update_counts(self, decrease_count=True, d, i, w):
		"""
		decrease_count: Boolean
		d: 		 		document number
		i:		 		ith word [unordered] in document d
		w:		 		term w in document d

		Updates count variables to run collapsed sampling equation for LDA.
		"""
		# We no longer decrement counts of N_{d,k}?
		if decrease_count:
			#self.local_count[d,z[(d,i)]] -= 1
			self.nkv[z[(d,i)],w] -= 1
		else:
			#self.local_count[d,self.z[(d,i)]] += 1
			self.nkv[self.z[(d,i)],w] += 1

	@staticmethod
	def roll_die_first_doc(n_d=local_count, n_g=global_count, n=n, M=M):
		# probabilities of picking one of s
		probs_old_draws = local_count[0, :] / (self.alpha + n)

		# probabilities of a new draw being set to s
		probs_new_draw_s = global_count[0,:] / (self.gamma + n)
		probs_new_draw_s *= (self.alpha / (self.alpha + n))

		# probability of a new draw from the base measure
		prob_new_from_base = (self.alpha / (self.alpha + M)) * (self.gamma / (self.gamma + M))
		
		probabilities = list(probs_old_draws) + list(probs_new_draw_s) + list(prob_new_from_base)
		draw = choice(len(probabilities), p=probabilities)

		return draw

	@staticmethod
	def roll_die_first_word(n_d=local_count, n_g=global_count, M=M, doc_idx = idx):
		# probabilities of picking one of s
		probs_old_draws = global_count[idx, :] / (self.gamma + M)

		# probability of a new draw
		prob_new_draw = self.gamma / (M + self.gamma)

		probabilities = list(probs_old_draws) + list(prob_new_draw)
		draw = choice(len(probabilities), p=probabilities)
		return draw

	@staticmethod
	def roll_die_nth_word(n_d=local_count, n_g=global_count, n=n, M=M, doc_idx=idx):
		# probabilities of picking one of s
		probs_old_draws = local_count[idx, :] / (self.alpha + n)

		# probability of a new draw
		prob_new_draw = self.alpha / (self.alpha + n)

		probabilities = list(probs_old_draws) + list(prob_new_draw)
		draw = choice(len(probabilities), p=probabilities)
		return draw

# Uncollapsed Gibbs

Here, I was more confused. Mainly, I haven't fully understood how drawing 'm' from a symmetric Dirichlet prior to drawing theta_d is equivalent to the above. In particular, where does the "rich get richer" effect would come from?

We draw 'm' in the initialization stage, then draw theta's and then topic assignments to words.

In further stages of the chain, I did not quite understand how we update 'm' conditionally on its previous draw:
    - In the vanilla LDA, the conditional posterior for theta was Dir(alpha + n_{d, :}) where n_{d, :} is a count vector of topics in document d. How does the drawing of 'm' beforehand affect this?
    
 

In [ ]:
    def _initialize(self, dtm):
		"""
		dtm: a DxV document-term matrix

		Initialises the count variables N_dk, N_kv, N_k, N_d.
			K: topic
			D: document
			V: term
		"""
		
		self.num_docs, self.vocab_size = dtm.shape

		# number of terms/words in document D that have topic allocation K
		self.ndk = np.zeros((self.num_docs, self.k))
		# number of times topic K allocaition variables generate term/word V
		self.nkv = np.zeros((self.k, vocab_size))
		# number of terms/words generated by topic K
		self.nk = np.zeros(self.k)

		# The Dirichlet prior for the document-topic distribution
        
        m = dirichlet(self.gamma * np.ones(self.k)) # m is corpus-wide
		theta = dirichlet(self.alpha * m, size = num_docs)

		# The Dirichlet prior for the topic-term distribution
		beta = dirichlet(self.eta * np.ones(self.vocab_size), size = self.num_topics)

		# Initialize the topic assignment variables as a dictionary
			# key corresponds to (d,n): nth word in document d
			# value = {0,...,K-1} is the topic assigned to that word
		self.z = {}
		for d in range(self.num_docs):
			for i, w in enumerate(DocTermMatrix.word_index(dtm[d,:])):
				# total iterations of i will equal the document length
				# the set of w includes all terms in document d
				self.z[(d,i)] = multinomial(1, theta[d]).nonzero()[0][0]
				self.ndk[d,self.z[(d,i)]] += 1
				self.nkv[self.z[(d,i)],w] += 1
				self.nk[self.z[(d,i)]] += 1

	def run_Gibbs(self, dtm, maxIter=100):
		"""
		Uncollapsed Gibbs sampling.

		matrix: the document-term matrix.
		maxIter: the number of iterations to run.

		One could construct predictive distributions for theta and beta from the posterior samples (like we do in the collapsed Gibbs).
		"""
		self._initialize(dtm)
		theta_posterior = theta 	# DxK matrix
		beta_posterior = beta 		# KxV matrix
		theta_samples = []
		beta_samples = []

																		# The number of loops seems very inefficient
		for iterr in range(maxIter):
			for k in range(self.num_topics):
				# Posterior for BETA
				beta_posterior[k,:] = dirichlet(self.eta * np.ones(self.k) + self.nkv[k,:])
			for d in range(self.num_docs):
				# Posterior for THETA
				theta_posterior[d,:] = dirichlet(self.alpha * m + self.ndk[d,:])
			for d in range(self.num_docs):
				for i, w in enumerate(DocTermMatrix.word_index(dtm[d,:])):
					self.update_counts(True, d, i, w)
					self.z[(d,i)] = self.sample_topic(d, w, theta_posterior, beta_posterior)
					# update counts
					self.update_counts(False, d, i, w)
			# Burn-in and thinning interval set to 10 iterations
			if (iterr + 1)%10 == 0:
				theta_samples.append(theta_posterior)
				beta_samples.append(beta_posterior)
			if iterr == maxIter:
				theta_samples.append(theta_posterior)
				beta_samples.append(beta_posterior)
																		
		#return z, theta_samples, beta_samples, self.ndk, self.nkv, self.nk 
		return theta_samples, beta_samples
    
	def sample_topic(self, d, w, theta, beta):
		zdi_probs = np.zeros(self.num_topics)
		for k in range(self.num_topics):
			zdi_probs[k] = theta[d,k]*beta[k,w] / np.dot(theta[d,:],beta[:,w])
		zdi_probs /= sum(zdi_probs)
		zdi = multinomial(1, zdi_probs).nonzero()[0][0]
		return zdi